In [72]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(model="meta-llama/Llama-4-Scout-17B-16E-Instruct")

# if the outputs for next cells are wrong, the free model may be overloaded. You can also use this public endpoint that contains Llama-3.2-3B-Instruct
# client = InferenceClient("https://jc26mwg228mkj8dw.us-east-1.aws.endpoints.huggingface.cloud")

In [ ]:
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of France is"},
    ],
    stream=False,
    max_tokens=1024
)
print(output.choices[0].message.content)

Paris.
